**About**
In this notebook, I created a model for predicting apartment prices in Mexico City. The goal is not to produce the best possible forecast, but rather to highlight the numerous phases required for creating such a model..

In [1]:
# Import libraries here
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt  
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.utils.validation import check_is_fitted
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder

Prepare Data

In [2]:
estate = pd.read_csv("/kaggle/input/real-estate/mexico-city-real-estate-1.csv")
estate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4628 entries, 0 to 4627
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   operation                   4628 non-null   object 
 1   property_type               4628 non-null   object 
 2   place_with_parent_names     4628 non-null   object 
 3   lat-lon                     4144 non-null   object 
 4   price                       4538 non-null   float64
 5   currency                    4538 non-null   object 
 6   price_aprox_local_currency  4538 non-null   float64
 7   price_aprox_usd             4538 non-null   float64
 8   surface_total_in_m2         1668 non-null   float64
 9   surface_covered_in_m2       4436 non-null   float64
 10  price_usd_per_m2            1150 non-null   float64
 11  price_per_m2                4249 non-null   float64
 12  floor                       291 non-null    float64
 13  rooms                       136 n

Data Wrangling

In [3]:
def wrangle(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # Subset data: Apartments in Mexico City ("Distrito Federal") that cost less than $100,000
    df = df[
        (df['place_with_parent_names'].str.contains("Distrito Federal")) &
        (df['price_aprox_usd'] < 100000) &
        (df['property_type'].str.lower() == "apartment")
    ]

    
    # Remove outliers by trimming bottom and top 10% of properties in "surface_covered_in_m2"
    Q1 = df['surface_covered_in_m2'].quantile(0.1)
    Q3 = df['surface_covered_in_m2'].quantile(0.9)
    IQR = Q3 - Q1
    df = df[(df["surface_covered_in_m2"] >= Q1) & (df["surface_covered_in_m2"] <= Q3)]
    #df['surface_covered_in_m2'] = df['surface_covered_in_m2'].clip(lower=Q1 - 1.5 * IQR, upper=Q3 + 1.5 * IQR)
    
    
     # Create separate "lat" and "lon" columns
    df['lat'] = df['lat-lon'].str.split(',', expand=False).str[0].astype(float)
    df['lon'] = df['lat-lon'].str.split(',', expand=False).str[1].astype(float)
    
    # Create "borough" feature from "place_with_parent_names"
    df['borough'] = df['place_with_parent_names'].str.split('|').str[1]
    
    # Drop columns with more than 50% null values
    null_columns = df.columns[df.isnull().any()].tolist()
    df = df.drop(columns=[col for col in null_columns if (df[col].isnull().sum() / len(df) > 0.5)])
     
    df['price_aprox_usd'] = df['price_aprox_usd'].fillna(0)  
    df= df.drop(columns = ['lat-lon','property_type', 'properati_url', 'price', 'currency', 'operation', 'place_with_parent_names', 'price_aprox_local_currency', 'price_per_m2'])
    
    return df

In [5]:
# Use this cell to test your wrangle function and explore the data
df2 = wrangle('/kaggle/input/real-estate/mexico-city-real-estate-1.csv')
df2.head()

,price_aprox_usd,surface_covered_in_m2,lat,lon,borough
11,94022.66,57.0,23.634501,-102.552788,Benito Juárez
20,70880.12,56.0,19.402413,-99.095391,Iztacalco
21,68228.99,80.0,19.357820,-99.149406,Benito Juárez
22,24235.78,60.0,19.504985,-99.208557,Azcapotzalco
26,94140.20,50.0,19.354219,-99.126244,Coyoacán


In [8]:
print("Shape:", df2.shape)

Shape: (1101, 5)


Using glob to create the list files

In [9]:
import glob

# Using glob to find all CSV files in the ./data directory except for mexico-city-test-features.csv
files = glob.glob("./kaggle/input/real-estate/mexico-city-real-estate-*.csv")
files = [file for file in files if "mexico-city-test-features.csv" not in file]

# Display the list of files
print(files)

[]


In [10]:
df = pd.concat((wrangle(file) for file in files), ignore_index=True)
print(df.info())
df.head()

ValueError: No objects to concatenate

EXPLORE the dataset

In [ ]:
# Build histogram
plt.hist(df["price_aprox_usd"])


# Label axes
plt.xlabel("Price [$]")
plt.ylabel("Count")

# Add title
plt.title("Distribution of Apartment Prices")

# Don't delete the code below 👇
plt.savefig("images/2-5-4.png", dpi=150)

In [ ]:
# Build scatter plot
plt.scatter(y = df["price_aprox_usd"], x = df["surface_covered_in_m2"])


# Label axes
plt.xlabel("Area [sq meters]")
plt.ylabel("Price [USD]")

# Add title
plt.title("Mexico City: Price vs. Area")

# Don't delete the code below 👇
plt.savefig("images/2-5-5.png", dpi=150)


In [ ]:
correlation = df['surface_covered_in_m2'].corr(df['price_aprox_usd'])
print(f'Correlation between Area and Price: {correlation}')

In [ ]:
# Plot Mapbox location and price
import plotly.express as px
fig = px.scatter_mapbox(df, 
                        lat="lat", 
                        lon="lon", 
                        color="price_aprox_usd",  # Color by price
                        color_continuous_scale="Viridis",  # Choose color scale
                        mapbox_style="open-street-map",
                        title="Real Estate Prices in Mexico City")
fig.show()

Split

In [ ]:
# Split data into feature matrix `X_train` and target vector `y_train`.

X_train = df.drop(columns=["price_aprox_usd"])
y_train = df["price_aprox_usd"]

Build the model

In [ ]:
y_mean = y_train.mean()
y_pred_baseline = [y_mean] * len(y_train)
print(y_pred_baseline[:5])
baseline_mae = mean_absolute_error(y_train, y_pred_baseline)
print("Mean apt price:", y_mean)
print("Baseline MAE:", baseline_mae)

In [ ]:
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge

# Build Model
model = make_pipeline(
        OneHotEncoder(use_cat_names=True),
        SimpleImputer(),
        Ridge(alpha=1.0)
)
# Fit model
model.fit(X_train, y_train)

In [ ]:
y_test_pred = pd.Series(model.predict(X_test))
y_test_pred.head()